In [1]:
!pip install -q datasets
!pip install -q sacrebleu
!pip install -q import-ipynb
!pip install -q tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard.writer import SummaryWriter
import torch.optim as optim
import warnings
from sacrebleu import corpus_bleu

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path
from typing import Any

from tqdm import tqdm

In [4]:
def get_config():
    return {
        "batch_size": 32,
        "num_epochs": 10,
        "lr": 2e-4,
        "seq_len": 570,
        "d_model": 512,
        "datasource": 'Helsinki-NLP/news_commentary',
        "lang_src": "en",
        "lang_tgt": "ru",
        "model_folder": "/content/drive/MyDrive/Colab Notebooks/Transformer",  # Путь в Colab
        "model_basename": "transformer_translate_model",
        "preload": None,
        "tokenizer_file": "tokenizer_{0}.json",
        "experimental_name": "runs/transformer_translate_model"
    }

def get_weights_file_path(config, epoch: str):
    model_folder = config['model_folder']
    model_basename = config['model_basename']
    model_filename = f"{model_basename}{epoch}.pt"
    return str(Path('.') / model_folder / model_filename)

In [5]:
class InputEmbeddings(nn.Module):
    # Инициализация слоя для встраивания входных данных
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model  # Размерность модели (выходной размер embedding)
        self.vocab_size = vocab_size  # Размер словаря
        # Слой embedding, который преобразует индексы слов в векторы размером d_model
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # Получаем embeddings и умножаем на корень из размерности модели для нормализации
        return self.embedding(x) * math.sqrt(self.d_model)


class PositionalEncoding(nn.Module):
    # Класс для добавления позиционного кодирования к входным данным
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model  # Размерность модели
        self.seq_len = seq_len  # Длина последовательности
        self.dropout = nn.Dropout(dropout)  # Слой dropout для регуляризации
        pe = torch.zeros(seq_len, d_model)  # Инициализация матрицы позиционного кодирования
        position = torch.arange(0, seq_len, dtype=torch.float32).unsqueeze(1)  # Позиции в последовательности
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # Нормировка
        pe[:, 0::2] = torch.sin(position * div_term)  # Синус для четных индексов
        pe[:, 1::2] = torch.cos(position * div_term)  # Косинус для нечетных индексов
        pe = pe.unsqueeze(0)  # Добавляем размерность для батча
        self.register_buffer('pe', pe)  # Регистрация буфера, чтобы не учитывать это при вычислении градиентов

    def forward(self, x):
        # Добавляем позиционное кодирование к входу
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x)  # Применяем dropout


class LayerNormalization(nn.Module):
    # Слой нормализации
    def __init__(self, eps: float = 1e-6):
        super().__init__()
        self.eps = eps  # Эпсилон для предотвращения деления на ноль
        self.alpha = nn.Parameter(torch.ones(1))  # Параметр масштабирования
        self.bias = nn.Parameter(torch.zeros(1))  # Параметр сдвига

    def forward(self, x):
        # Нормализация по последней оси (для каждого элемента последовательности)
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.alpha * (x - mean) / (std + self.eps) + self.bias


class FeedForward(nn.Module):
    # Простой полносвязный слой с активацией ReLU и dropout
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)  # Первый линейный слой
        self.dropout = nn.Dropout(dropout)  # Dropout
        self.linear2 = nn.Linear(d_ff, d_model)  # Второй линейный слой

    def forward(self, x):
        x = self.dropout(torch.relu(self.linear1(x)))  # Применяем ReLU и dropout
        x = self.linear2(x)  # Пропускаем через второй линейный слой
        return x


class MultiHeadAttentionBlock(nn.Module):
    # Модуль многоголовой внимательности
    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model  # Размерность модели
        self.h = h  # Количество голов в механизме внимания
        assert d_model % h == 0, "d_model must be divisible by h"  # Проверка, что размерность делится на количество голов
        self.d_k = d_model // h  # Размерность каждого подпространства внимания
        self.w_q = nn.Linear(d_model, d_model)  # Линейный слой для получения queries
        self.w_k = nn.Linear(d_model, d_model)  # Линейный слой для получения keys
        self.w_v = nn.Linear(d_model, d_model)  # Линейный слой для получения values
        self.w_o = nn.Linear(d_model, d_model)  # Линейный слой для проекции выходных данных
        self.dropout = nn.Dropout(dropout)  # Dropout

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        # Вычисление внимания с маской
        d_k = query.shape[-1]
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)  # Скалярное произведение и нормализация
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)  # Применяем маску
        attention_scores = attention_scores.softmax(dim=-1)  # Softmax для вычисления вероятностей
        if dropout is not None:
            attention_scores = dropout(attention_scores)  # Применяем dropout
        return (attention_scores @ value), attention_scores  # Возвращаем результаты внимания

    def forward(self, q, k, v, mask):
        # Применение внимания с несколькими головами
        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)
        # Перемещаем размерности для многоголового внимания
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)
        # Вычисляем внимание
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        # Возвращаем проекцию через линейный слой
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        return self.w_o(x)


class ResidualConnection(nn.Module):
    # Остаточная связь с нормализацией и dropout
    def __init__(self, dropout: float):
        super().__init__()
        self.dropout = nn.Dropout(dropout)  # Dropout
        self.norm = LayerNormalization()  # Нормализация

    def forward(self, x, sublayer):
        # Возвращаем сумму входных данных и выходных данных с субслоем, применяя нормализацию и dropout
        return x + self.dropout(sublayer(self.norm(x)))


class EncoderBlock(nn.Module):
    # Блок энкодера, включающий внимание и feed-forward слой
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForward, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block  # Модуль самовнимания
        self.feed_forward_block = feed_forward_block  # Модуль feed-forward
        self.residual_connection = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])  # Остаточные связи

    def forward(self, x, src_mask):
        # Применяем остаточные связи и слои
        x = self.residual_connection[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connection[1](x, self.feed_forward_block)
        return x


class Encoder(nn.Module):
    # Модуль энкодера, который состоит из нескольких слоев энкодеров
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers  # Слои энкодера
        self.norm = LayerNormalization()  # Нормализация

    def forward(self, x, mask):
        # Проходим через все слои энкодера
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)  # Возвращаем нормализованные данные


class DecoderBlock(nn.Module):
    # Блок декодера, включающий самовнимание, перекрестное внимание и feed-forward слой
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForward, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block  # Модуль самовнимания
        self.cross_attention_block = cross_attention_block  # Модуль перекрестного внимания
        self.feed_forward_block = feed_forward_block  # Модуль feed-forward
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])  # Остаточные связи

    def forward(self, x, memory, tgt_mask, memory_mask):
        # Применяем остаточные связи и слои
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, memory, memory, memory_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x


class Decoder(nn.Module):
    # Модуль декодера, состоящий из нескольких слоев декодеров
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers  # Слои декодера
        self.norm = LayerNormalization()  # Нормализация

    def forward(self, x, memory, tgt_mask, memory_mask):
        # Проходим через все слои декодера
        for layer in self.layers:
            x = layer(x, memory, tgt_mask, memory_mask)
        return self.norm(x)  # Возвращаем нормализованные данные

class ProjectionLayer(nn.Module):
    # Проекционный слой, который преобразует выходные данные в вероятности для каждого слова в словаре
    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        # Линейный слой, который преобразует вектор размерности d_model в вектор размерности vocab_size
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # Применяем softmax по последней оси (для каждого слова) для получения вероятностей
        return torch.log_softmax(self.proj(x), dim=-1)


class Transformer(nn.Module):
    # Основной класс трансформера, который включает энкодер, декодер и проекционный слой
    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings,
                 tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding,
                 projection_layer: ProjectionLayer):
        super().__init__()
        # Инициализируем все компоненты трансформера
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # Кодируем входную последовательность (источник)
        src = self.src_embed(src)  # Применяем embedding для исходных данных
        src = self.src_pos(src)  # Добавляем позиционное кодирование
        return self.encoder(src, src_mask)  # Пропускаем через энкодер

    def decode(self, encoder_output, src_mask, tgt, tgt_mask):
        # Декодируем цельную последовательность (цель)
        tgt = self.tgt_embed(tgt)  # Применяем embedding для целевых данных
        tgt = self.tgt_pos(tgt)  # Добавляем позиционное кодирование
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)  # Пропускаем через декодер

    def project(self, x):
        # Применяем проекционный слой для перевода выходных данных в логарифмические вероятности
        return self.projection_layer(x)


def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int,
                      tgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8,
                      dropout: float = 0.1, d_ff: int = 2048) -> Transformer:
    # Строим трансформер с указанными параметрами
    src_embed = InputEmbeddings(d_model, src_vocab_size)  # Слой для исходных эмбеддингов
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)  # Слой для целевых эмбеддингов
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)  # Позиционное кодирование для исходных данных
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)  # Позиционное кодирование для целевых данных

    # Создаем блоки энкодера
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)  # Многоголовое внимание для энкодера
        feed_forward_block = FeedForward(d_model, d_ff, dropout)  # Feed-forward блок
        encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)  # Блок энкодера
        encoder_blocks.append(encoder_block)

    # Создаем блоки декодера
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)  # Многоголовое внимание для декодера
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)  # Перекрестное внимание
        feed_forward_block = FeedForward(d_model, d_ff, dropout)  # Feed-forward блок
        decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)  # Блок декодера
        decoder_blocks.append(decoder_block)

    # Создаем энкодер и декодер
    encoder = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))

    # Создаем проекционный слой
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    # Создаем трансформер
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Инициализируем параметры трансформера
    for p in transformer.parameters():
        if p.dim() > 1:  # Инициализируем параметры с использованием Xavier инициализации для слоев с матрицами
            nn.init.xavier_uniform_(p)

    return transformer  # Возвращаем построенный трансформер


In [6]:
# Определение датасета для двуязычного текста
class BiLanguageDataset(Dataset):
    # Инициализация датасета, где будет храниться информация для пар исходных и целевых предложений
    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len) -> None:
        super().__init__()
        # Сохраняем параметры, переданные при инициализации
        self.ds = ds  # Датасет с парами исходных и целевых предложений
        self.tokenizer_src = tokenizer_src  # Токенизатор для исходного языка
        self.tokenizer_tgt = tokenizer_tgt  # Токенизатор для целевого языка
        self.src_lang = src_lang  # Исходный язык
        self.tgt_lang = tgt_lang  # Целевой язык
        self.seq_len = seq_len  # Максимальная длина последовательности
        # Определяем специальные токены для начала ([SOS]), конца ([EOS]) и заполнителя ([PAD])
        self.sos_token = torch.tensor([tokenizer_src.token_to_id('[SOS]')], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_src.token_to_id('[EOS]')], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_src.token_to_id('[PAD]')], dtype=torch.int64)

    # Возвращаем количество элементов в датасете
    def __len__(self):
        return len(self.ds)

    # Возвращаем один элемент из датасета (пар исходного и целевого текста)
    def __getitem__(self, index: Any) -> Any:
        # Получаем пару исходного и целевого текста
        src_target_pair = self.ds[index]
        src_text = src_target_pair['translation'][self.src_lang]  # Исходный текст
        tgt_text = src_target_pair['translation'][self.tgt_lang]  # Целевой текст

        # Токенизируем тексты в индексы токенов
        enc_input_tokens = self.tokenizer_src.encode(src_text).ids  # Токены для исходного текста
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids  # Токены для целевого текста

        # Вычисляем количество токенов для заполнения, чтобы привести длину до seq_len
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2  # Минус 2 из-за SOS и EOS токенов
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1  # Минус 1 для EOS

        # Проверка на слишком длинные предложения
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError('Sentence is too long')  # Выбрасываем ошибку, если предложение слишком длинное

        # Собираем вход для энкодера: добавляем SOS, токены текста, EOS и заполняем до seq_len
        encoder_input = torch.cat(
            [
                self.sos_token,  # Начало исходной последовательности
                torch.tensor(enc_input_tokens, dtype=torch.int64),  # Токены исходного текста
                self.eos_token,  # Конец исходной последовательности
                torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64)  # Заполнение до seq_len
            ]
        )

        # Собираем вход для декодера: добавляем SOS, токены целевого текста и заполняем до seq_len
        decoder_input = torch.cat(
            [
                self.sos_token,  # Начало целевой последовательности
                torch.tensor(dec_input_tokens, dtype=torch.int64),  # Токены целевого текста
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)  # Заполнение до seq_len
            ]
        )

        # Собираем метки (label) для декодера: токены целевого текста, EOS и заполняем до seq_len
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),  # Токены целевого текста
                self.eos_token,  # Конец последовательности
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)  # Заполнение до seq_len
            ]
        )

        # Проверка на правильность размеров всех последовательностей
        assert encoder_input.size(0) == self.seq_len  # Должен быть одинаковый размер
        assert decoder_input.size(0) == self.seq_len  # Должен быть одинаковый размер
        assert label.size(0) == self.seq_len  # Должен быть одинаковый размер

        # Возвращаем словарь с подготовленными данными
        return {
            'encoder_input': encoder_input,  # Вход для энкодера
            'decoder_input': decoder_input,  # Вход для декодера
            'encoder_mask': (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),  # Массив маски для энкодера
            'decoder_mask': (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int() & causal_mask(decoder_input.size(0)),  # Массив маски для декодера
            'label': label,  # Метки для декодера
            'src_text': src_text,  # Исходный текст
            'tgt_text': tgt_text  # Целевой текст
        }

# Функция для создания маски для декодера, где каждый токен может только "видеть" предыдущие
def causal_mask(size):
    # Создаем верхнюю треугольную матрицу (маску)
    mask = torch.triu(torch.ones(1, size, size), diagonal=1).type(torch.int)
    return mask == 0  # Все элементы выше главной диагонали становятся 0, остальные - 1


In [7]:
# Функция для получения модели
def get_model(config, src_vocab_size, tgt_vocab_size):
    # Строим модель с помощью конфигурации и размеров словаря исходного и целевого языков
    model = build_transformer(src_vocab_size, tgt_vocab_size, config['seq_len'], config['seq_len'], d_model=config['d_model'])
    return model

# Функция для загрузки данных
def get_ds(config):
    # Загружаем датасет из источника данных, указанный в конфигурации
    ds_raw = load_dataset(config['datasource'], f"{config['lang_src']}-{config['lang_tgt']}", split='train')

    # Загружаем токенизаторы для исходного и целевого языка
    tokenizer_src = Tokenizer.from_file(config['tokenizer_file'].format(config['lang_src']))
    tokenizer_tgt = Tokenizer.from_file(config['tokenizer_file'].format(config['lang_tgt']))

    # Разделяем датасет на обучающую и валидационную выборки (70% на обучение, 30% на валидацию)
    train_ds_size = int(0.7 * len(ds_raw))  # 70% для обучения
    val_ds_size = len(ds_raw) - train_ds_size  # 30% для валидации

    # Разделение на обучающую и валидационную выборки
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

    # Создаем экземпляры датасетов для обучения и валидации
    train_ds = BiLanguageDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
    val_ds = BiLanguageDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

    # Создаем загрузчики данных для обучения и валидации с батчами
    train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)  # shuffle=True для случайного перемешивания данных
    val_dataloader = DataLoader(val_ds, batch_size=config['batch_size'], shuffle=False)  # shuffle=False для валидации

    # Возвращаем даталоадеры и токенизаторы
    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt

# Функция для валидации модели
def run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, seq_len, device, print_msg, global_step, writer, num_examples=2):
    model.eval()  # Переводим модель в режим оценки (без обучения)
    with torch.no_grad():  # Отключаем вычисления градиентов (память и скорость)
        # Перебираем батчи из валидационного датасета
        for batch in val_dataloader:
            # Перемещаем данные в устройство (GPU или CPU)
            encoder_input = batch['encoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)
            decoder_input = batch['decoder_input'].to(device)
            decoder_mask = batch['decoder_mask'].to(device)
            label = batch['label'].to(device)

            # Пропускаем вход через энкодер
            output = model.encode(encoder_input, encoder_mask)
            # Пропускаем через декодер
            output = model.decode(output, encoder_mask, decoder_input, decoder_mask)
            # Прогнозируем выход (логарифм вероятности для каждого токена)
            output = model.project(output)

            # Печатаем сообщение о текущем шаге валидации
            print_msg(f"Validation step {global_step}")

            # Прерываем цикл после первого батча (для примера)
            break


In [8]:
def translate_sentence(model, src_text, tokenizer_src, tokenizer_tgt, seq_len, device):
    # Переводим модель в режим оценки (без обучения)
    model.eval()

    # Отключаем вычисление градиентов для ускорения работы
    with torch.no_grad():
        # Токенизируем исходный текст с помощью токенизатора для исходного языка
        src_tokens = tokenizer_src.encode(src_text).ids
        # Преобразуем токены в тензор и добавляем дополнительную размерность для батча
        src = torch.tensor(src_tokens, dtype=torch.int64).unsqueeze(0).to(device)

        # Создаем маску для исходных данных (1 - если токен не [PAD], иначе 0)
        src_mask = (src != tokenizer_src.token_to_id('[PAD]')).unsqueeze(0).unsqueeze(0).int().to(device)

        # Пропускаем исходный текст через энкодер модели
        encoder_output = model.encode(src, src_mask)

        # Инициализируем токены для начала декодирования с символа [SOS]
        tgt = torch.tensor([tokenizer_tgt.token_to_id('[SOS]')], dtype=torch.int64).unsqueeze(0).to(device)

        # Начинаем генерировать целевой текст по очереди
        for _ in range(seq_len):
            # Создаем маску для декодера, которая предотвращает "заглядывание" в будущие токены
            tgt_mask = causal_mask(tgt.size(1)).to(device)  # Функция causal_mask должна быть определена

            # Пропускаем декодируемые данные через модель (декодер)
            output = model.decode(encoder_output, src_mask, tgt, tgt_mask)

            # Получаем логарифм вероятности для каждого токена
            output = model.project(output)

            # Выбираем токен с наибольшей вероятностью для следующего шага
            next_token = output.argmax(dim=-1)[:, -1].item()

            # Добавляем следующий токен к целевому вводу
            tgt = torch.cat([tgt, torch.tensor([[next_token]], dtype=torch.int64).to(device)], dim=1)

            # Прерываем цикл, если встретился токен окончания предложения [EOS]
            if next_token == tokenizer_tgt.token_to_id('[EOS]'):
                break

        # Преобразуем сгенерированные токены в текст
        translated_tokens = tgt.squeeze().tolist()
        translated_text = tokenizer_tgt.decode(translated_tokens)

        return translated_text


In [9]:
# Основной код
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = get_config()

# Получаем данные и токенизаторы
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)

# Создаём модель
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# Загружаем веса модели
model_filename = f"{config['model_folder']}/{config['model_basename']}09.pt"
state = torch.load(model_filename, map_location=device)
model.load_state_dict(state['model_state_dict'])
model.eval()

# Запускаем валидацию
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None)

# Пример перевода
references = []  # Эталонные переводы
hypotheses = []  # Предсказанные переводы
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for batch in tqdm(val_dataloader, desc="Processing batches"):
    src_text = batch['src_text'][0]  # Исходный текст
    tgt_text = batch['tgt_text'][0]  # Эталонный перевод
    translated_text = translate_sentence(model, src_text, tokenizer_src, tokenizer_tgt, config['seq_len'], device)
    references.append(tgt_text)
    hypotheses.append(translated_text)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/45.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/190104 [00:00<?, ? examples/s]

<ipython-input-9-44f5bb76b79c>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(model_filename, map_location=device)


Validation step 0


Processing batches: 100%|██████████| 1783/1783 [11:34<00:00,  2.57it/s]


In [10]:
# 7. Вычисление BLEU
bleu_score = corpus_bleu(hypotheses, [references])
print(f"BLEU Score: {bleu_score.score}")


BLEU Score: 34.22474095037862


BLEU Score: 34.22474095037862

In [11]:
# 8. Пример вывода для проверки
for i in tqdm(range(5), desc="Printing examples"):  # Прогресс-бар для примеров
    print(f"Исходный текст: {references[i]}")
    print(f"Эталонный перевод: {hypotheses[i]}")
    print("-" * 50)

Printing examples: 100%|██████████| 5/5 [00:00<00:00, 13469.18it/s]

Исходный текст: Правительствам, которым придется решать задачу повышения расходов, необходимо будет разработать целевые системы социальной защиты, которые защитят их бедных людей за счет выделения денежных средств или квазиденежных трансфертов.
Эталонный перевод: Правительства , которые сталкиваются с большими расходами , должны разработать целевые системы социальной защиты , которые защищают своих бедных людей посредством денежных и денежных переводов .
--------------------------------------------------
Исходный текст: Нашей работой по экономическому развитию, сохранению мира, охране окружающей среды и здравоохранению, мы помогаем странам и сообществам строить лучшее, более свободное и процветающее будущее.
Эталонный перевод: Нашей работой в развитии , сохранении мира , охране окружающей среды и здоровье , мы странам и сообществам строить лучшее , более будущее .
--------------------------------------------------
Исходный текст: Поскольку население из отказывающейся от евро страны про

Printing examples: 100%|██████████| 5/5 [00:00<00:00, 13469.18it/s]Исходный текст: Правительствам, которым придется решать задачу повышения расходов, необходимо будет разработать целевые системы социальной защиты, которые защитят их бедных людей за счет выделения денежных средств или квазиденежных трансфертов.
Эталонный перевод: Правительства , которые сталкиваются с большими расходами , должны разработать целевые системы социальной защиты , которые защищают своих бедных людей посредством денежных и денежных переводов .
--------------------------------------------------
Исходный текст: Нашей работой по экономическому развитию, сохранению мира, охране окружающей среды и здравоохранению, мы помогаем странам и сообществам строить лучшее, более свободное и процветающее будущее.
Эталонный перевод: Нашей работой в развитии , сохранении мира , охране окружающей среды и здоровье , мы странам и сообществам строить лучшее , более будущее .
--------------------------------------------------
Исходный текст: Поскольку население из отказывающейся от евро страны продолжало бы хранить евро, уход из Европейского экономического и валютного союза не снизил бы уровень существующего благосостояния.
Эталонный перевод: Поскольку люди из страны могут продолжать держать евро , оставляя ЕВС не приведет к потере существующего благосостояния .
--------------------------------------------------
Исходный текст: Подтвержден��е того, что эти реликвии холодной войны устарели как средства обеспечения безопасности – и, что на самом деле они совсем не безопасны - исходит от разных сторон.
Эталонный перевод: , что эти холодной войны устарели как инструменты безопасности – и , действительно , они вызывают нестабильность – исходит от разнообразных рядов голосов .
--------------------------------------------------
Исходный текст: Мы можем иметь очень поверхностную информацию о компаниях, которые стоят за всеми этими удобствами и получают нашу информацию; но, помимо маркетинговой информации, сотрудники этих компаний являются для нас безликими и безымянными.
Эталонный перевод: У нас может быть понимание того , что компании , за этими , выполняют наши данные ; но помимо маркетинга , реальные люди , которые управляют этими организациями , и .
--------------------------------------------------

In [12]:
import pandas as pd

# Создаём DataFrame из hypotheses и references
df = pd.DataFrame({'Hypothesis': hypotheses, 'Reference': [ref[0] for ref in references]})

# Сохраняем DataFrame в CSV
df.to_csv('translations_dataframe.csv', index=False, encoding='utf-8')
